# 金融市场情绪专题
PCA

In [1]:
import sys
import os
import cudf
import pandas as pd

# ------------------------------数据集路径----------------------------------#
# /usr/local/miniconda3/envs/Rapids/bin/python
DATASETS_PATH = '/data/DataSets/BW_INDEX_CSMAR/BW_5_CSMAR/'

In [2]:
os.listdir(DATASETS_PATH)

['QX_STOCKRATE[DES][csv].txt',
 'QX_TRM1.csv',
 'QX_FUNDDISCOUNTPREMIUM[DES][csv].txt',
 'QX_DIVIDENDPREMIUM.csv',
 'QX_FUNDDISCOUNTPREMIUM1.csv',
 'QX_FUNDDISCOUNTPREMIUM.csv',
 'QX_DIVIDENDPREMIUM[DES][csv].txt',
 'QX_FUNDDISCOUNTPREMIUM.parquet',
 'QX_TRM.csv',
 'QX_TRM[DES][csv].txt',
 'QX_STOCKRATE.csv',
 'QX_IPO[DES][csv].txt',
 'QX_IPO.csv']

### 股利溢价-年
SgnYear [统计年度] - YYYY
Dnum [分红公司数目] - 指当年发生了现金分红的上市公司数量
NDnum [未分红公司数目] - 指当年没有现金分红的上市公司数量
DMB [分红公司平均市值账面比] - 计算公式为：所有分红的公司的市值账面比的平均数
NDMB [未分红公司平均市值账面比] - 计算公式为：所有未分红的公司的市值账面比的平均数
PDND [股利溢价] - 计算公式为：分红公司平均市值账面比-未分红公司平均市值账面比
LogPDND [对数股利溢价] - 计算公式为：股利溢价的对数

In [3]:
# pd.read_csv(f'{DATASETS_PATH}QX_DIVIDENDPREMIUM.csv')

### 股票融资占股票债券融资比例-年

In [4]:
# pd.read_csv(f'{DATASETS_PATH}QX_STOCKRATE.csv')

### 基金折溢价率CEFD-面板
Symbol [基金代码] -
TradingDate [交易日期] -
CategoryID [基金类别] - S0601=股票型基金;S0602=债券型基金;S0603=货币型基金;S0604=混合型基金;S0605=FOF;S0606=股指期货型基金;S0699=其他
FundTypeID [基金运作方式] - S0501=契约型开放式;S0502=契约型封闭式
IsETF [是否ETF] - 1=是；2=否
IsLOF [是否LOF] - 1=是；2=否
IsIndexFund [是否指数基金] - 1=是；2=否
IsInnovative [是否创新型基金] - 1=是；2=否
NAV [基金份额净值] -
AccumulativeNAV [基金份额累计净值] -
ClosePrice [收盘价] -
CovertRate [溢价率(%)] - 计算公式为：（收盘价-基金份额净值）*101/基金份额净值
TurnoverRate [换手率] -

In [14]:

def cal_cefd():
    """
    计算封闭式基金溢价
    """
    
    '''
    保留 封闭式基金
    剔除 LOF基金,创新型基金,股票指数基金
    '''

    df = (
            cudf.from_pandas(
                    cudf.read_parquet(f'{DATASETS_PATH}QX_FUNDDISCOUNTPREMIUM.parquet').sort_index().to_pandas()
                    .query(" FundTypeID=='S0502'& IsLOF==2 & IsInnovative==2 & IsIndexFund==2")
            )
    )

    '''
    平均加权CovertRate
    '''

    df['DayAvgCR'] = df.groupby(level=['TradingDate'])['CovertRate'].transform('mean')

    # CovertRate
    return df


df_cefd = cal_cefd()
df_cefd

CategoryID FundTypeID  IsETF  IsLOF  IsIndexFund  \
TradingDate Symbol                                                    
19980430    184688      S0601      S0502      2      2            2   
            500001      S0601      S0502      2      2            2   
19980630    184688      S0601      S0502      2      2            2   
            500001      S0601      S0502      2      2            2   
            500003      S0601      S0502      2      2            2   
...                       ...        ...    ...    ...          ...   
20221222    501098      S0604      S0502      2      2            2   
            501099      S0604      S0502      2      2            2   
            501200      S0604      S0502      2      2            2   
            501201      S0604      S0502      2      2            2   
            501202      S0604      S0502      2      2            2   

                    IsInnovative     NAV  AccumulativeNAV  ClosePrice  \
TradingDate Symbol                                                      
19980430    184688             2  1.0000           1.0000       2.020   
            500001             2  1.0448           1.0448       2.100   
19980630    184688             2  1.0442           1.0442       1.680   
            500001             2  1.0409           1.0409       1.680   
            500003             2  1.0184           1.0184       1.580   
...                          ...     ...              ...         ...   
20221222    501098             2  1.0215           1.0215       1.024   
            501099             2  1.3172           1.3172       1.316   
            501200             2  0.7318           0.7318       0.815   
            501201             2  1.5362           1.5362       1.544   
            501202             2  0.9975           0.9975       0.980   

                    CovertRate TurnoverRate    DayAvgCR  
TradingDate Symbol                                       
19980430    184688      102.00         <NA>  101.500000  
            500001      101.00         <NA>  101.500000  
19980630    184688       60.89         <NA>   60.272500  
            500001       61.40         <NA>   60.272500  
            500003       55.15         <NA>   60.272500  
...                        ...          ...         ...  
20221222    501098        0.24     0.001429    0.293333  
            501099       -0.09     0.001591    0.293333  
            501200       11.37     0.003242    0.293333  
            501201        0.51     0.002085    0.293333  
            501202       -1.75      0.00039    0.293333  

[35712 rows x 12 columns]

### 市场换手率-面板数据
TradingDate [交易日期] -
MarketType [市场类型] - 1=上证A股市场；2=上证B股市场；4=深证A股市场；8=深证B股市场；6=主板；5=沪深A股市场；7=中小板；10=沪深B股市场；15=沪深AB股市场；16=创业板；20=深证A股和创业板；21=沪深A股和创业板；31=沪深AB股和创业板；32=科创板；33=上证A股和科创板；37=沪深A股和科创板；47=沪深AB股和科创板；53=沪深A股和创业板和科创板；63=沪深AB股和创业板和科创板；64=北证A股市场；69=沪深京A股市场；79=沪深京AB股市场；85=沪深京A股和创业板；95=沪深京AB股和创业板；101=沪深京A股和科创板；111=沪深京AB股和科创板；117=沪深京A股和创业板和科创板；127=沪深京AB股和创业板和科创板；
Parameter [参数值] - 字段说明见说明书“附录 参数说明”
Unit [参数单位] -
TurnoverRate1 [换手率(总股本)] - 计算公式为：交易量/总股本
TurnoverRate2 [换手率(流通股本)] - 计算公式为：交易量/流通股本

In [6]:
# pd.read_csv(f'{DATASETS_PATH}QX_TRM.csv')

### IPO股票信息-面板数据
Symbol [股票代码] -
ListedDate [上市日期] -
ListedYear [上市年份] -
ABSymbol [AB股交叉码] - 指公司同时发行了A和B股的情况
HSymbol [H股交叉码] - 指的是公司同时发行了H股的情况
EstablishDate [公司成立日期] -
IpoDate [首次招股日期] -
CompanyListedDate [公司上市日期] -
IsIPO [是否首次发行] - A=是；B=否
TotalShares [实际发行总量] -
IssuePrice [发行价格] -
CurrencyCode [发行价格币种] -
ClosePrice [上市首日收盘价] -
ReturnRate [上市首日回报率] -
TurnoverRate1 [上市首日换手率(总股本)] -
TurnoverRate2 [上市首日换手率(流通股本)] -
PE [上市首日市盈率] -
PB [上市首日市净率] -

In [7]:
# pd.read_csv(f'{DATASETS_PATH}QX_IPO.csv')